# Model with Binary Data

Example testing a model which uses binary data
 

In [1]:
!cp ../../../proto/prediction.proto ./proto
!cp -r ../../../proto/tensorflow/tensorflow .
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

In [2]:
import json
import requests
import base64
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import numpy as np
import pickle

def get_payload():
    array_2d = np.arange(9).reshape(3, 3)
    print(array_2d)
    bdata = pickle.dumps(array_2d)
    return bdata

In [35]:
def rest_request_ambassador(deploymentName,namespace,request,endpoint="localhost:8003"):
    response = requests.post(
                "http://"+endpoint+"/seldon/"+namespace+"/"+deploymentName+"/api/v0.1/predictions",
                json=request)
    print(response.status_code)
    print(response.text)
    return response.json()

def grpc_request_ambassador(deploymentName,namespace,data,endpoint="localhost:8004"):
    request = prediction_pb2.SeldonMessage(binData = data)
    channel = grpc.insecure_channel(endpoint)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('seldon',deploymentName),('namespace',namespace)]
    response = stub.Predict(request=request,metadata=metadata)
    return response

def rest_request_docker(request,endpoint="localhost:5000"):
    response = requests.post(
                "http://"+endpoint+"/predict",
                 data={"json":json.dumps(request),"isDefault":True})
    print(response.text)
    return response.json()

def grpc_request_docker(data,endpoint="localhost:5000"):
    request = prediction_pb2.SeldonMessage(binData = data)
    channel = grpc.insecure_channel(endpoint)
    stub = prediction_pb2_grpc.ModelStub(channel)
    response = stub.Predict(request=request)
    return response


## REST

In [11]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python36:0.13 model-with-bindata-rest:0.1

---> Installing application source...
Build completed successfully


In [12]:
!docker run --name "model-with-bindata" -d --rm -p 5000:5000 model-with-bindata-rest:0.1

1af22a1967d507a92e773ad7ee5d195a5a8197fed4e7b4ac7a2fe7995da045e6


In [14]:
bdata = get_payload()
bdata_base64 = base64.b64encode(bdata).decode('utf-8')
payload = {"meta":{},"binData":bdata_base64}
response = rest_request_docker(payload)
bdata2 = base64.b64decode(response["binData"])
arr_resp = pickle.loads(bdata2)
print(arr_resp)
print(arr_resp.shape)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
{"binData":"gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShLAUsDSwOGcQZjbnVtcHkKZHR5cGUKcQdYAgAAAGk4cQhLAEsBh3EJUnEKKEsDWAEAAAA8cQtOTk5K/////0r/////SwB0cQxiiUNIAAAAAAAAAAABAAAAAAAAAAIAAAAAAAAAAwAAAAAAAAAEAAAAAAAAAAUAAAAAAAAABgAAAAAAAAAHAAAAAAAAAAgAAAAAAAAAcQ10cQ5iLg==","meta":{"metrics":[{"key":"mycounter","value":1.0},{"key":"mygauge","type":"GAUGE","value":100.0},{"key":"mytimer","type":"TIMER","value":20.200000762939453}]}}

[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)


In [15]:
!docker rm model-with-bindata --force

model-with-bindata


## gRPC

In [16]:
!s2i build -E environment_grpc . seldonio/seldon-core-s2i-python36:0.13 model-with-bindata-grpc:0.1

---> Installing application source...
Build completed successfully


In [17]:
!docker run --name "model-with-bindata" -d --rm -p 5000:5000 model-with-bindata-grpc:0.1

046eb6370f931c9fc7f41066d43d2dbb7d6a7da3372cf59b85c8cc936d73a5bd


In [18]:
payload = get_payload()

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [19]:
bdata = get_payload()
resp = grpc_request_docker(bdata)
bdata2 = resp.binData
arr_resp = pickle.loads(bdata2)
print(arr_resp)
print(arr_resp.shape)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)


In [20]:
!docker rm model-with-bindata --force

model-with-bindata


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --vm-driver kvm2 --memory 4096 

😄  minikube v1.0.0 on linux (amd64)
🤹  Downloading Kubernetes v1.14.0 images in the background ...
💡  Tip: Use 'minikube start -p <name>' to create a new cluster, or 'minikube delete' to delete this one.
🏃  Re-using the currently running kvm2 VM for "minikube" ...
⌛  Waiting for SSH access ...
📶  "minikube" IP address is 192.168.39.225
🐳  Configuring Docker as the container runtime ...


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

# REST

In [30]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python36:0.13 model-with-bindata-rest:0.1

---> Installing application source...
Build completed successfully


In [31]:
!kubectl create -f deployment-rest.json

seldondeployment.machinelearning.seldon.io/mymodel created


In [32]:
!kubectl rollout status deploy/mymodel-mymodel-b0e3779

deployment "mymodel-mymodel-b0e3779" successfully rolled out


## Test predict

In [37]:
minikube_ip = !minikube ip
minikube_port = !kubectl get svc -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].spec.ports[0].nodePort}'

In [40]:
bdata = get_payload()
bdata_base64 = base64.b64encode(bdata).decode('utf-8')
payload = {"meta":{},"binData":bdata_base64}
response = rest_request_ambassador("mymodel","default",payload,minikube_ip[0]+":"+minikube_port[0])
bdata2 = base64.b64decode(response["binData"])
arr_resp = pickle.loads(bdata2)
print(arr_resp)
print(arr_resp.shape)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
200
{
  "meta": {
    "puid": "ocvn7n6p5654vvj2epvrmfd7mg",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "complex-model": "model-with-bindata-rest:0.1"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 100.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 20.2,
      "tags": {
      }
    }]
  },
  "binData": "gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShLAUsDSwOGcQZjbnVtcHkKZHR5cGUKcQdYAgAAAGk4cQhLAEsBh3EJUnEKKEsDWAEAAAA8cQtOTk5K/////0r/////SwB0cQxiiUNIAAAAAAAAAAABAAAAAAAAAAIAAAAAAAAAAwAAAAAAAAAEAAAAAAAAAAUAAAAAAAAABgAAAAAAAAAHAAAAAAAAAAgAAAAAAAAAcQ10cQ5iLg=="
}
[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)


In [41]:
!kubectl delete -f deployment-rest.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


# gRPC

In [42]:
!eval $(minikube docker-env) && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python36:0.13 model-with-bindata-grpc:0.1

---> Installing application source...
Build completed successfully


In [43]:
!kubectl create -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [44]:
!kubectl rollout status deploy/mymodel-mymodel-a9ecaa4

deployment "mymodel-mymodel-a9ecaa4" successfully rolled out


In [45]:
bdata = get_payload()
response = grpc_request_ambassador("mymodel","default",bdata,minikube_ip[0]+":"+minikube_port[0])
bdata2 = response.binData
arr_resp = pickle.loads(bdata2)
print(arr_resp)
print(arr_resp.shape)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)


In [46]:
!kubectl delete -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


In [ ]:
!minikube delete